In [1]:
import requests
import bs4
import time
import pandas as pd
import re

In [2]:
FILENAME='verbs.csv'

In [3]:
verbs = pd.read_csv('hebrew_verbs.csv', sep = ';')

In [4]:
error_msg='BANNED'
# pause = 0.1

In [5]:
# links = []

In [6]:
# n = 0
# for verb in verbs['Verb']:
#     if str(verb) != 'nan':
#         n += 1
#         req = requests.get('https://www.pealim.com/search/?q='+verb)
#         if req.status_code == 521:
#             print(error_msg)
#             print(n)
#             break
# #         time.sleep(pause)
#         parser = bs4.BeautifulSoup(req.text, 'lxml')
#         div = parser.find('div',attrs={'class' : 'verb-search-lemma'})
#         if div:
#             for a in div.children:
#                 link = a.get('href')
#                 if link:
#                     links.append(link)
#                 else:
#                     print('no link - ' + verb)
#         else:
#             print(verb)

In [7]:
# links = pd.DataFrame(links, columns=['Link'])
# links.to_csv('links.csv')

In [15]:
links = pd.read_csv('links.csv', sep = ',')

In [16]:
vowel_pattern = re.compile(r"[\u0591-\u05C7]")
def remove_vowels(text: str) -> str:
    return re.sub(vowel_pattern, "", text)

In [17]:
res = []

In [18]:
for link in links['Link']:
    req = requests.get('https://www.pealim.com' + link)
    if req.status_code == 521:
        print(error_msg)
        print(link)
        break
    verb_description = []
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    p = parser.find('b')
    verb_description.append('https://www.pealim.com' + link)
    verb_description.append(p.text)
    div = parser.find('div',attrs={'class' : 'lead'})
    verb_description.append(div.text)
    tds =  parser.findAll('td',attrs={'class' : 'conj-td'})
    for td in tds:
        div = td.find('span',attrs={'class' : 'menukad'})
        verb_description.append(remove_vowels(div.text))
        verb_description.append(div.text)
    res.append(verb_description)
    

In [19]:
res = pd.DataFrame(res)
res.to_csv('res.csv',sep = ';')

In [20]:
res2 = pd.read_csv('res.csv', sep = ';')

In [21]:
res2

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,95,96,97,98,99,100,101,102,103,104
0,0,https://www.pealim.com/dict/1-lichtov/,PA'AL,to write,כותב,כּוֹתֵב,כותבת,כּוֹתֶבֶת,כותבים,כּוֹתְבִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://www.pealim.com/dict/38-lirkod/,PA'AL,to dance,רוקד,רוֹקֵד,רוקדת,רוֹקֶדֶת,רוקדים,רוֹקְדִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://www.pealim.com/dict/1441-laavor/,PA'AL,to pass; to overtake; to move,עובר,עוֹבֵר,עוברת,עוֹבֶרֶת,עוברים,עוֹבְרִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://www.pealim.com/dict/468-lahafoch/,PA'AL,"to turn over, to turn into (ל-)",הופך,הוֹפֵךְ,הופכת,הוֹפֶכֶת,הופכים,הוֹפְכִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://www.pealim.com/dict/128-leesof/,PA'AL,"to collect, to pick up, to reap",אוסף,אוֹסֵף,אוספת,אוֹסֶפֶת,אוספים,אוֹסְפִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,https://www.pealim.com/dict/220-lehitbalbel/,HITPA'EL,"to become confused, perplexed",מתבלבל,מִתְבַּלְבֵּל,מתבלבלת,מִתְבַּלְבֶּלֶת,מתבלבלים,מִתְבַּלְבְּלִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,130,https://www.pealim.com/dict/1245-lehitnatzel/,HITPA'EL,to apologize,מתנצל,מִתְנַצֵּל,מתנצלת,מִתְנַצֶּלֶת,מתנצלים,מִתְנַצְּלִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,131,https://www.pealim.com/dict/1508-lehitamel/,HITPA'EL,"to exercise, to work out",מתעמל,מִתְעַמֵּל,מתעמלת,מִתְעַמֶּלֶת,מתעמלים,מִתְעַמְּלִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,132,https://www.pealim.com/dict/1947-lehitkarev/,HITPA'EL,"to approach, to come near",מתקרב,מִתְקָרֵב,מתקרבת,מִתְקָרֶבֶת,מתקרבים,מִתְקָרְבִים,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
